# Molecular dynamics hands-on Python coding practices

This Jupyter Notebook file serves as a gateway to enter the realm of Python programming and molecular dynamics. We'll break down the basic MD loop to several code cells, step-by-step. The main purpose of this coding assignment is to horn your Python programming skills. The first part will be creating multiple arrays, as containers, to store atoms' information such as positions, velocity, acceleration, interatomic relative position vector, forces, etc. The second part will be run time iterations to calculate atomic foreces, accelerations, and update atomic positions. Note that the material parameters are arbitrarily set up. You'll need to use realistic parameters if you want to simulate a real system.

by Hui-Chia Yu, 2025

---


#### <p style="text-align: right;"> &#9989; **Put your name here** </p>

---
### Part 1: build containers to store data

#### Part 1.1: Initial atomic positions, velocities, and accelerations
* First, we'll create three containers to store atoms' postion vectors, velocity vectors, and acceleration vectors. See the illustraion below. We'll use numpy array for these containers. (Python has many different contrainers; for example, list, dictionary, tuple, set, etc. They have diferent properties and usages. We're not going over them today.) Numpy arrays are created to mimic the arrays in Matlab and they could be used in a similar way. 
* Python is an open-source software package. There are tons of libraries that are created by other developers and they are shared openly via PyPI system. Those libraries are made following standard software sturcture and documentation practices. New users can go to resspective git site and git page to get familiar with the software libraries. For example, [https://numpy.org.](https://numpy.org)

<div align="center">
<img src="https://i.ibb.co/HDcbFjKm/arrays.png" width="260">
</div>

In [8]:
# Let's set up materials parameters, such as atomic mass, epsilon, etc.
# These parameters are set up arbitrarily. For real MD simulations,
# you will need to input realistic materials parameters.

ma = 0.5         # <-- mass
rm = 0.1         # <-- reference radius
eps = 0.25       # <-- coefficient for energy well
r0 = rm*1.12246  # <-- equilibrium radius


# compuational box size
Lx = 1.0
Ly = 1.0


# we first import numpy library
import numpy as np
import random

# set up the number of atoms
N = 64

# create position, velocity, and acceleration arrays
Pos = np.zeros((N,2))
Vel = np.random.rand(N, 2)  # <-- random initial velocity
Vel *= 0.001 
Acc = np.random.rand(N, 2)  # <-- random initial acceleration 
Acc *= 0.0001 

# set up initial atom positions. Let's make a 8x8 configuration
for i in range(8):           # <-- note that default Python loop goes from 0 to n-1
    for j in range(8):       # <-- note indent is what Python recognizes nest loop
        Pos[i*8+j,0] = j*r0 + 0.75*r0 + random.uniform(-0.01, 0.01) # <-- add some randomness
        Pos[i*8+j,1] = i*r0 + 0.75*r0 + random.uniform(-0.01, 0.01)


# save the initial atomic positions
Pos_pr = Pos.copy()
Vel_pr = Vel.copy()
Acc_pr = Acc.copy()

**Let's visualize the initial atom configuration in the cell below.**

In [ ]:
# import matplotlib library, which is built to
# mimic Matlab's plotting functionality

import matplotlib.pyplot as plt  

# Plot results
# plt.scatter(Pos[:, 0], Pos[:, 1], c='b')
plt.plot(Pos[:, 0], Pos[:, 1], 'bo')
plt.xlabel("x Position")
plt.ylabel("y Position")
plt.title("Particle Positions")
plt.gca().set_aspect('equal', adjustable='box')
plt.axis([0, Lx, 0, Ly])

#### Part 1.2: Calculate and store relative vectors and distances between every atom pair.

* **Let's create a 3d array to store the relative position vectors between atom pairs.** Since we have N atoms, we'll make a NxNx3 array; for instance, a `dst[i,j,k]` array, where we use index i for the atom of interest, index j for the neighboring atoms, k = 0 and 1 for the $x$ component ($\Delta x_{ij}$) and $y$ component ($\Delta y_{ij}$) of the relative vector, respectively, and k = 2 for the distance between two atoms ($r_{ij}$).
$$\Delta x_{ij} = x_j - x_i,~~~~\Delta x_{ji} = -\Delta x_{ij}$$
$$\Delta y_{ij} = y_j - y_i,~~~~\Delta y_{ji} = -\Delta y_{ij}$$

* For periodic cells, we need to consider the mirroring neighbor atoms from the neighboring cell. See the figure below for illustration.
<div align="center">
<img src="https://i.ibb.co/60vv5VJ8/MD-Instruction-Note1-A.jpg" width="280">
</div>
Along the $x$ direction, we'll pick the closest neighboring atom using $$\Delta x_{ij} = \min(|\Delta x_{ij}|, |\Delta x_{ij}+L_x|, |\Delta x_{ij}-L_x|).$$
Similarly, in the $y$ direction, we'll take $$\Delta y _{ij} = \min(|\Delta y_{ij}|, |\Delta y_{ij}+L_y|, |\Delta y_{ij}-L_y|)$$ for the closest atom (in the surrounding repeated cell).

* The distance between two atoms is calculated using:
$$r_{ij} = \sqrt{\Delta x_{ij}^2 + \Delta y_{ij}^2},~~~~r_{ji} =r_{ij}.$$

* Since atoms are relative to each other in opposite direction. Thus, we only need to calculate half of the number of atom pairs. For example, calculate the upper right triagnle of the whole data set and mirror the values, with a negative sign, to the lower left triangle. See the figure below for illustration.
<div align="center">
<img src="https://i.ibb.co/hSW4t4Z/MD-Instruction-Note-array1-A.jpg" width="800">
</div>
  

**Create the distance array as described above in the code cell below.**


In [ ]:
# create a 3d array to store relative positions between atom pairs.
import math

# fill the ?? in the code
dst = np.zeros((??,??,??))

# loop over atoms and neighbors to calculate interatomic distances
for at in range(N-1):
    for ne in range(at+1,N):  # <-- calculate only the upper right triangle
        dx_c = ??
        dx_w = dx_c - Lx
        dx_e = dx_c + Lx
        # x component
        dst[at,ne,0] = min([dx_c, dx_w, dx_e], key=abs)

        dy_c = ??
        dy_s = ??
        dy_n = ??
        # y component
        dst[at,ne,1] = min([dy_c, dy_s, dy_n], key=abs)

        # distance between 2 atoms
        dst[at,ne,2] = math.sqrt(dst[??,??,??]**2 + dst[??,??,??]**2)

        # lower left triangle
        dst[ne,at,0] = -dst[??,??,0]
        dst[ne,at,1] = -dst[??,??,1]
        dst[ne,at,2] =  dst[??,??,2]            



#### Part 1.3: Build arrays to store interatomic forces and calculate the quantities

* We then make another 3d array, with the size of NxNx2, to store interatomic forces between atom pairs. Let's use `Fij[i,j,k]`, where k = 0 and 1 are for the $x$ and $y$ components of the forces. See the illustraion below.
  
<div align="center">
<img src="https://i.ibb.co/HLdnWWpf/MD-Instruction-Note-Fij.jpg" width="580">
</div>

* The interatomic forces are calculated using $$\vec{f}_{ij} = -\frac{\vec{r}_{ij}}{r_{ij}}  \frac{24 \varepsilon}{r_m} \bigg[2 \bigg(\frac{r_m}{r_{ij}}\bigg)^{13} - \bigg(\frac{r_m}{r_{ij}}\bigg)^7 \bigg],$$ where $\vec{r}_{ij} = (\Delta x_{ij}, \Delta y_{ij}).$

  
* We will create anthor 2d array of Nx2 size to store the vector of the total force on each atom. This array will be similar to the position-vector array created earlier. Let's use `Fi_v[i,j]`, where i is the label of atoms, and j is the index for $x$ or $y$ components. The total force vector is obtained by summing the respective row in the $\vec{f}_{ij}$ array:
$$\vec{f}_i = \sum_{j}^N\vec{f}_{ij}.$$ See the figure below for illustration. 

* Here, we also set a cutoff radius to be $r_{cut} = 3r_0$, outside the cutoff range, the interatomic forces are ignored.

* Again, we only need to calculate the upper right triangle and mirror the values to the low triangle of the data array.

In [ ]:
# build Fij and Fi_v array, and calculate the values of the entries.

# fill the ?? in the code

Fij = np.zeros((??))
r_cut = 3*r0

# loop over atoms and neighbors to calculate interatomic forecs
Fij.fill(0)
for at in range(N-1):
    for ne in range(at+1,N):
        if 0 < ?? < r_cut:

            dUdr = (24*eps/rm)*(2*(??)**13 - (??)**7)
            # x component
            Fij[at,ne,0] = -dst[at,ne,0]/dst[at,ne,2]*dUdr
            Fij[ne,at,0] = -Fij[at,ne,0]
            
            # y component
            Fij[at,ne,1] = ??
            Fij[ne,at,1] = ??            


# sum up the interatomic forces to get the total force on the atom of interest.
Fi_v = np.zeros((N,2))
Fi_v.fill(0)
for at in range(N):
    Fi_v[at,0] = ??
    Fi_v[at,1] = ??


When you arrive this part of the code, all the necessary containers should be successfully created.

---
### Part 2: Time stepping

#### Part 2.1
In this part, we'll put those arrays above to make the time stepping simulations to update atoms' positions iteratively. Following the procedure,

* Calculate interatomic distances between pairs.
* Calculate interatomic forces between pairs. Note we only calculate those atoms within the cutoff radius ($r_{cut}$). We keep the cutoff radius to be three times equilibrium distance ($r_{cut} = 3r_0 = 3\times 1.12246 \cdot r_m$). For neighbors outside that range, the interaction forces are zero.
* Calculate total force (`Fi_v`) of each atom by summing each row of the `Fij` array.
* Use Newton's law to calculate acceleration of each atom: $$\vec{a}_i = \frac{\vec{f}_i}{m}.$$
* We will try a simple time method first. Use Euler method:
$$\vec{r}(t_{n+1}) = \vec{r}(t_{n})+ \vec{v}(t_{n})\cdot \Delta t$$
$$\vec{v}(t_{n+1}) = \vec{v}(t_{n})+ \vec{a}(t_{n})\cdot \Delta t$$ to update atoms positions and velocities.
* Consider the periodic boundary conditions: If an atom leaves the computational box from one boundary, it must re-enter the box from the opposite boundary. See the figure below for illustration.
<div align="center">
<img src="https://i.ibb.co/gbHx7PMs/MD-Instruction-Note7.jpg" width="260">
</div>
$$\text{if} ~~ x_{i}(t+\Delta t) < 0, ~~\text{then} ~~x_{i}(t+\Delta t) = x_{i}(t+\Delta t)  + L_x$$
$$\text{if} ~~ x_{i}(t+\Delta t) > L_x, ~~\text{then} ~~x_{i}(t+\Delta t) = x_{i}(t+\Delta t) - L_x$$
$$\text{if} ~~ y_{i}(t+\Delta t) < 0, ~~\text{then} ~~y_{i}(t+\Delta t) = y_{i}(t+\Delta t)  + L_y$$
$$\text{if} ~~ y_{i}(t+\Delta t) > L_y, ~~\text{then} ~~y_{i}(t+\Delta t) = y_{i}(t+\Delta t) - L_y$$
    

**Use the code cell to make the time simulation.**

In [ ]:
# import libraries for plotting
from IPython.display import display, clear_output
import time

# Initialization before the loop
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)

# set up number of time steps and time step size
nstep = 10001
dt = 0.00005
r_cut = 3*r0


# get initial positions from Part 1
Pos = Pos_pr.copy() 
Vel = Vel_pr.copy()
Acc = Acc_pr.copy()


# fill the ?? in the code below


tm = 0
cnt = 1

# Simulation loop
for it in range(0, nstep + 1):

    # Periodic boundary conditions
    for at in range(N):
        # in x direction
        if Pos[at,0] < 0:
            Pos[at,0] = Pos[at,0] + Lx
        if Pos[at,0] > Lx:
            Pos[at,0] = ??

        # in y direction
        if Pos[at,1] < 0:
            Pos[at,1] = ??
        if Pos[at,1] > Lx:
            Pos[at,1] = ??
            
    # Calculate inter-atom distances with periodic boundaries
    for at in range(N - 1):
        for ne in range(at + 1, N):
            dx_c = ??
            dx_w = ??
            dx_e = ??

            dy_c = ??
            dy_s = ??
            dy_n = ??

            # X component
            dst[at,ne,0] = min([dx_c, dx_w, dx_e], key=abs)
            dst[ne,at,0] = -dst[at,ne,0]

            # Y component
            dst[at,ne,1] = min([dy_c, dy_s, dy_n], key=abs)
            dst[ne,at,1] = -dst[at,ne,1]

            # Inter-atom distance
            dst[at,ne,2] = np.sqrt(??)
            dst[ne,at,2] = dst[at,ne,2]

    # Calculate force
    Fij.fill(0)

    for at in range(N - 1):
        for ne in range(at + 1, N):
            if 0 < dst[at,ne,2] < r_cut:
                dUdr = ??
                
                Fij[at,ne,0] = ??
                Fij[at,ne,1] = ??

                Fij[ne,at,0] = -Fij[at,ne,0]
                Fij[ne,at,1] = -Fij[at,ne,1]

    # Calculate total force of each atom
    for at in range(N):
        Fi_v[at,0] = ??
        Fi_v[at,1] = ??
    
    # calculate acceleration of each atom
    # Acc = Fi_v/ma
    for at in range(N): 
        Acc[at,0] = Fi_v[at,0]/ma
        Acc[at,1] = Fi_v[at,1]/ma   
        
    # Update positions
    for at in range(N): 
        Pos[at,0] += Vel[at,0] * dt
        Pos[at,1] += Vel[at,1] * dt

    
    # Update velocity
    for at in range(N): 
        Vel[at, 0] += Acc[at, 0] * dt
        Vel[at, 1] += Acc[at, 1] * dt

    # print(X_Y)

    # Elapsed time
    tm += dt

    # Visualization
    if it % 50 == 1:
        plt.plot(Pos[:, 0], Pos[:, 1], 'bo')
        plt.axis([0, Lx, 0, Ly])
        print(it)
   
        # Animaiton part (dosn't change)
        clear_output(wait=True) # Clear output for dynamic display
        display(fig)            # Reset display
        fig.clear()             # Prevent overlapping and layered plots
        time.sleep(0.0002)         # Sleep for half a second to slow down the animation

---

#### Part 2.2

Now, **let's use Verlet time scheme.**

* At the initial step: $$\vec{r}(t_1) = \vec{r}(t_0) + \vec{v}(t_0)\cdot \Delta t + \frac{1}{2}\vec{a}(t_0)\cdot \Delta t^2.$$ Now, since there are position arrays: one for $t_1$ and the other for $t_0$, we'll need to create another position vector to store the position in the previous time step.

* * Use Verlet's method to update atomic positions: $$\vec{r}(t_{n+1}) = 2\vec{r}(t_{n}) - \vec{r}(t_{n-1}) + \vec{a}(t_{n})\cdot \Delta t^2  $$

In [ ]:

# import libraries for plotting
from IPython.display import display, clear_output
import time

# Initialization before the loop
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)


# set up number of time steps and time step size
nstep = 10001
dt = 0.00005
r_cut = 3*r0


# get initial positions from Part 1
Pos = Pos_pr.copy() 
Vel = Vel_pr.copy()
Acc = Acc_pr.copy()

for at in range(N):
    Pos[at,0] = Pos_pr[at,0] + Vel[at,0]*dt + 0.5*Acc[at,0]*dt**2
    
Pos_c = Pos.copy()
Pos_p = Pos_pr.copy()

tm = 0
cnt = 1

# copy your functioning code in the cell above to the respective part of 
# this code.
# Simulation loop
for it in range(0, nstep + 1):

    # Periodic boundary conditions
 
            
    # Calculate inter-atom distances with periodic boundaries
 

    # Calculate force
    Fij.fill(0)

    for at in range(N - 1):
        for ne in range(at + 1, N):
            if 0 < dst[at,ne,2] < r_cut:


    # Calculate total force of each atom
    for at in range(N):

    
    # calculate acceleration of each atom
    # Acc = Fi_v/ma
    for at in range(N): 
        Acc[at,0] = Fi_v[at,0]/ma
        Acc[at,1] = Fi_v[at,1]/ma   
        
    # Update positions
    for at in range(N): 
        Pos[at,0] = 2*?? - ?? + Acc[at,0]*dt**2
        Pos[at,1] = 2*?? - ?? + Acc[at,1]*dt**2

    Pos_p = Pos_c.copy()
    Pos_c = Pos.copy()

    # print(X_Y)

    # Elapsed time
    tm += dt

    # Visualization
    if it % 50 == 1:
        plt.plot(Pos[:, 0], Pos[:, 1], 'bo')
        plt.axis([0, Lx, 0, Ly])
        print(it)

   
        # Animaiton part (dosn't change)
        clear_output(wait=True) # Clear output for dynamic display
        display(fig)            # Reset display
        fig.clear()             # Prevent overlapping and layered plots
        time.sleep(0.0002)         # Sleep for half a second to slow down the animation




**Upload you final copy to the drop box on the course website. Lab 1 coding assignment.**
DO NOT forget adding your name to the file name.